_Run the first 2 code cells without modifications_

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
#pd.set_option("display.max_colwidth", 200)
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-v0_8-muted')
from pprint import pprint as ptp

import sys
from pathlib import Path
print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """

    if up:
        newp = str(Path(this_path).parent)
    else:
        newp = str(Path(this_path))
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


def fdir(obj, start_with_str='_', exclude=True):
    """Filtered dir() for method discovery."""
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]


def md_width_comment(w:int=120) -> str:
    """Width guide for composing md documents."""
    return f"<!-- dotted line width = {w}\n{'.'*w}-->"


# autoreload extension
%load_ext autoreload
%autoreload 2

Python ver: 3.11.5 | packaged by conda-forge | (main, Aug 27 2023, 03:34:09) [GCC 12.3.0]
Python env: mce
Currrent dir: /home/cat/projects/MCCE_Benchmarking/notebooks



In [2]:
add_to_sys_path(Path.cwd(), up=True)
notebooks_dir = Path.cwd()
notebooks_dir

Path added to sys.path: /home/cat/projects/MCCE_Benchmarking


PosixPath('/home/cat/projects/MCCE_Benchmarking/notebooks')

---
---

### To get time as per MG's outline:
Do this cmd via suprocess in each of the proteins folders & gather -> all_run_times.txt
```
grep 'Total time' run.log
```
### Version:

#### Version from run.log:
```
grep -m1 'Version' run.log
>>Version:        1.1.0
```

#### Version from exec:
This will not work for conda-installed mcce (as per Quick Start docs):
```
which mcce
>>/home/mcce/Stable-MCCE/bin/mcce
```
```
grep 'VERSION' /home/mcce/Stable-MCCE/lib/*.h
>>#define VERSION       "MCCE2.5.1"
```


In [30]:
txt = """
<div>
    <p>First text</p>
    <p>Second text</p>
</div>
<div>
    <p>Second text</p>
    <p>Third text</p>
</div>
"""

!printf "%s\n" "$txt"


<div>
    <p>First text</p>
    <p>Second text</p>
</div>
<div>
    <p>Second text</p>
    <p>Third text</p>
</div>


Error: no such file ""


In [29]:
!printf "%s\n" a '<p>' c d '<p>' '<p>' '<p>' e

!printf "%s\n" a '<p>' c d '<p>' '<p>' '<p>' e | sed "0,/<p>/!{//d}"

a
<p>
c
d
<p>
<p>
<p>
e
a
<p>
c
d
e


In [32]:
from mcce_benchmark import BENCH, DEFAULT_DIR, APP_NAME, MCCE_EPS, N_SLEEP, N_ACTIVE, CRON_COMMENT, ENTRY_POINTS, USER
from mcce_benchmark import audit, job_setup
import subprocess
print(BENCH)

2024-02-06 11:09:22,326 @cat [INFO: mcce_benchmark]: - START
----------------------------------------------------------------------
APP VER: (0, 1, 'dev37', 'g3fc647b.d20240202')
APP DEFAULTS:
    Globals: MCCE_EPS = 4; N_SLEEP = 10; N_ACTIVE = 10
    Default names:
    DEFAULT_DIR = 'mcce_benchmarks'
    BENCH.CLEAN_PDBS = 'clean_pdbs'
    BENCH.Q_BOOK = 'book.txt'
    BENCH.DEFAULT_JOB = 'default_run'
----------------------------------------------------------------------


        BENCH_DATA = /home/cat/projects/MCCE_Benchmarking/mcce_benchmark/data
        BENCH_WT = /home/cat/projects/MCCE_Benchmarking/mcce_benchmark/data/WT_pkas.csv
        BENCH_PROTS = /home/cat/projects/MCCE_Benchmarking/mcce_benchmark/data/proteins.tsv
        BENCH_PDBS = /home/cat/projects/MCCE_Benchmarking/mcce_benchmark/data/clean_pdbs
        DEFAULT_JOB = default_run
        DEFAULT_JOB_SH = /home/cat/projects/MCCE_Benchmarking/mcce_benchmark/data/clean_pdbs/default_run.sh
        BENCH_Q_BOOK = /home/ca

In [33]:
# in my ~/projects folder:
benchmarks_dir = Path.cwd().parent.parent.joinpath(DEFAULT_DIR)
benchmarks_dir

PosixPath('/home/cat/projects/mcce_benchmarks')

---
## Scheduling

In [129]:
from mcce_benchmark.scheduling import subprocess_run, build_cron_path, build_cron_cmd, create_crontab

In [138]:
PA = build_cron_path()
print(PA)
launch_cmd = build_cron_cmd(benchmarks_dir, "foo", 4,"step2_out.pdb")
print(launch_cmd)


cron = CronTab(user=True)
for job in cron:
    print(job)

cron.remove_all(comment=CRON_COMMENT)

create_crontab(PA, launch_cmd)

#Scheduled from mccebench
PATH=/home/cat/miniconda3/bin:/home/cat/miniconda3/envs/mce/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin

#Scheduled from mccebench
* * * * * cd /home/cat/projects/mcce_benchmarks && /home/cat/miniconda3/envs/mce/bin/mccebench_launchjob -benchmarks_dir /home/cat/projects/mcce_benchmarks -job_name foo -n_active 4 -sentinel_file step2_out.pdb



0

[INFO]: mcce_benchmark.scheduling, create_crontab:
	Crontab text:
#Scheduled from mccebench
PATH=/home/cat/miniconda3/bin:/home/cat/miniconda3/envs/mce/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin
#Scheduled from mccebench
* * * * * cd /home/cat/projects/mcce_benchmarks && /home/cat/miniconda3/envs/mce/bin/mccebench_launchjob -benchmarks_dir /home/cat/projects/mcce_benchmarks -job_name foo -n_active 4 -sentinel_file step2_out.pdb



no crontab for cat


[INFO]: mcce_benchmark.scheduling, create_crontab:
	User's cron jobs, if any:


In [139]:
for job in cron:
    print(job)
    # Iterate over all lines:
for line in cron.lines:
    print(line)

---
### Future Job Setup?

---
# Test cli

In [38]:
from mcce_benchmark import cli

In [41]:
cli_parser = cli.bench_parser()
cli_parser.print_usage()

usage: 
mccebench <+ sub-command :: one of [data_setup, script_setup, launch_batch]> <related args>

Examples for current implementation (Beta):

1. Data setup
 - Using defaults (benchmarks_dir= mcce_benchmarks):
   >mccebench data_setup

 - Using a different folder name:
   >mccebench data_setup -benchmarks_dir <different name>

2. Script setup
 - Using defaults (benchmarks_dir= mcce_benchmarks; job_name= default_run):
   >mccebench script_setup

 - Using non-default option(s):
   >mccebench script_setup -job_name <my_job_name>
   >mccebench script_setup -benchmarks_dir <different name> -job_name <my_job_name>

3. Submit batch of jobs
 - Using defaults (benchmarks_dir= mcce_benchmarks;
                   job_name= default_run;
                   n_active= 10;
                   sentinel_file= pK.out):
   >mccebench launch_batch

 - Using non-default option(s):
   >mccebench launch_batch -n_active <jobs to maintain>
   >mccebench launch_batch -job_name <my_job_name> -sentinel_file step

In [37]:
print(cli_parser.description)


Description:
Launch a MCCE benchmarking job using curated structures from the pKa Database v1.

The main command is 'mccebench' along with one of 3 sub-commands:
- Sub-command 1: 'data_setup': setup data folders;
- Sub-command 2: 'script_setup': setup the run script to run mcce steps 1 through 4;
- Sub-command 3: 'launch_batch': launch a batch of jobs;

Post an issue for all errors and feature requests at:
https://github.com/GunnerLab/MCCE_Benchmarking/issues




In [13]:
# script setup: includes deletion of 'sentinel file' (i.e. pK.out or step2_out.pdb)

job = "foo"
cmd = f"script_setup -benchmarks_dir {benchmarks_dir} -job_name {job}".split()
cmd
args = cli_parser.parse_args(cmd)
args.func(args)

['script_setup',
 '-benchmarks_dir',
 '/home/cat/projects/mcce_benchmarks',
 '-job_name',
 'foo']

[INFO]: mcce_benchmark.cli, bench_script_setup:
	mccebench args:
{'subparser_name': 'script_setup',
 'benchmarks_dir': PosixPath('/home/cat/projects/mcce_benchmarks'),
 'job_name': 'foo',
 'dry': False,
 'norun': False,
 'e': 'mcce',
 'eps': 4,
 'u': '',
 'func': <function mcce_benchmark.cli.bench_script_setup(args: argparse.Namespace) -> None>}

[INFO]: mcce_benchmark.cli, bench_script_setup:
	Write fresh book file.
[INFO]: mcce_benchmark.cli, bench_script_setup:
	Writing script for 'foo' job.
[INFO]: mcce_benchmark.job_setup, get_default_script:
	Re-installed default_run.sh
[INFO]: mcce_benchmark.job_setup, write_run_script:
	Soft-linked default_run.sh as foo.sh
[INFO]: mcce_benchmark.job_setup, write_run_script:
	Script contents:
#!/bin/bash

step1.py --dry prot.pdb
step2.py -d 4
step3.py -d 4
step4.py

sleep 10

[INFO]: mcce_benchmark.cli, bench_script_setup:
	Deleting previous pK.out files, if any.
[INFO]: mcce_benchmark.job_setup, delete_pkout:
	0 pK.out file(s) deleted.


## NEW req: conda env required (for scheduling)
-> MOVE check_env() to `__ini__.py`

In [45]:
# MOVE to __ini__.py

# Check the conda env in use:
def check_env() -> Union[None,str]:
    pref = sys.prefix
    if "envs" not in pref:
        # NO_GO
        raise ImplementationError("A dedicated conda env is required.")
    return Path(sys.prefix).name

False

'mce'

In [24]:
# launch_batch: now required the name of the conda env where the cli was installed

mc_env = check_env()
n_jobs_to_maintain = 3

cmd = f"launch_batch -conda_env {mc_env} -benchmarks_dir {benchmarks_dir} -job_name {job} -n_active {n_jobs_to_maintain}".split()
cmd
args = cli_parser.parse_args(cmd)
args.func(args)

['launch_batch',
 '-benchmarks_dir',
 '/home/cat/projects/mcce_benchmarks',
 '-job_name',
 'dummy',
 '-n_active',
 '3']

[INFO]: mcce_benchmark.cli, bench_launch_batch:
	mccebench args:
{'subparser_name': 'launch_batch',
 'benchmarks_dir': PosixPath('/home/cat/projects/mcce_benchmarks'),
 'job_name': 'dummy',
 'n_active': 3,
 'sentinel_file': 'pK.out',
 'func': <function mcce_benchmark.cli.bench_launch_batch(args: argparse.Namespace) -> None>}

[INFO]: mcce_benchmark.cli, bench_launch_batch:
	Submiting batch of jobs.
[INFO]: mcce_benchmark.cli, bench_launch_batch:
	Debug mode: batch_submit.launch_job not called


---
---
---
# Packaged data management
## Prep of the "master" pdbs folder, `BENCH_PDBS`:
 * Remove any MCCE output files or folder along with prot.pdb
---

## tests

In [ ]:
import random

In [ ]:
matched_pks = []
for i in range(10):
    matched_pks.append((random.choice("ABCDRGWSX"),
                         random.choice([3.2, 5.1, 6., 4.4, 7.2]),
                        random.choice([3.2, 5.1, 6., 4.4, 7.2]*2)))
matched_pks

In [ ]:
pka_dict = experimental_pkas_to_dict(WT)
len(pka_dict)
list(pka_dict.keys())[:10]